## Francony Alexandre

# WebScrapper for MTG decks on cardmarket.com

Nous allons dans un premier temps nous assurer que le site est atteignable à l'aide de la librairie requests et de la fonction get_status_code. Cette fonction sera réutilisée dans le code pour vérifier que le site est toujours atteignable avant chaque étape de récupération majeure.

In [1]:
import requests
import csv
import os
import requests
from bs4 import BeautifulSoup
import re

def get_status_code(url):
    response = requests.get(url)
    return response.status_code

url = "https://www.cardmarket.com/fr/Magic/Products/Singles/Phyrexia-All-Will-Be-One"
status_code = get_status_code(url)
print("Le code d'état de Cardmarket est", status_code)


Le code d'état de Cardmarket est 200


In [2]:
def prep(card):
    #transformer tout les caractères non alphanumériques en tirets et tout en minuscule et les espaces en tirets
    return re.sub(r'\W+', '-', card).lower()

In [22]:
with open('data/deck.txt', 'r') as file:
    decklist = file.read()

#Récupérer la première ligne du fichier qui contient 'MIN_QUALITY = "<qualité>"'
MIN_QUALITY = decklist.splitlines()[0].split(" = ")[1].strip('"')
#print(MIN_QUALITY)

#Récupérer chaque autre ligne du fichier qui contient "quantité nom de la carte" et les stocker dans un dictionaire
deck = {}
for line in decklist.splitlines()[1:]:
    quantity, card = line.split(" ", 1)
    deck[card] = quantity
#print(deck)

#et maintenant ajoutons l'url de chaque carte dans le dictionnaire
for card in deck:
    deck[card] = [deck[card], "https://www.cardmarket.com/en/Magic/Cards/" + prep(card)]
#print(deck)


In [30]:
qualities = []
def switch_case(argument):
    switcher = {
        "MT": ["MT"],
        "NM": ["NM", "MT"],
        "EX": ["EX", "NM", "MT"],
        "GD": ["GD", "EX", "NM", "MT"],
        "LP": ["LP", "GD", "EX", "NM", "MT"],
        "PL": ["PL", "LP", "GD", "EX", "NM", "MT"],
        "PO": ["PO", "PL", "LP", "GD", "EX", "NM", "MT"],
    }
    return switcher.get(argument, ["MT"])

qualities = switch_case(MIN_QUALITY)
#print(qualities)


In [45]:
def getCardName(card):
    #récupérer le nom de la carte qui est la clé du dictionnaire
    card_name = card
    if(card_name == ''):
        card_name = 'Undefined'
    return card_name;

def getCardPrices(card):
    cards_sales = []
    tab_prices = []
    
    # Récupération du contenu de la page
    url = deck[card][1]
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Recherche de la liste des cartes
    table = soup.find_all('div', {'class': 'row no-gutters article-row'})

    for row in table:
        cards_sales.append(row)

    #récupérer les 5 premiers prix de vente de la carte
    for card in cards_sales:
        card_price = card.find('span', {"class" : "font-weight-bold color-primary small text-right text-nowrap"}).text.strip()[:-2]
        tab_prices.append(card_price.replace(',', '.'))
        if len(tab_prices) == 5:
            break

    return tab_prices;

def getCardQuantity(card):
    # récupérer l'url de la carte
    url = deck[card][1]
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Récupération du cadre contenant les informations de la carte
    card_frame = soup.find('dl', {"class" : "labeled row no-gutters mx-auto"})

    # Récupération de la quantité de la carte
    frame_infos = card_frame.find_all('dd', {"class" : "col-6 col-xl-7"})[-6:]

    # récupérer seulement la quantité, qui est contenu dans le premier élément du tableau
    card_quantity = frame_infos[0].text.strip()

    return card_quantity;

def getCardAveragePrice(card):
    # Récupération du contenu de la page
    url = deck[card][1]
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Récupération du cadre contenant les informations de la carte
    card_frame = soup.find('dl', {"class" : "labeled row no-gutters mx-auto"})

    # Récupération des données en chiffre de la carte
    frame_infos = card_frame.find_all('dd', {"class" : "col-6 col-xl-7"})[-6:]
    
    # récupérer seulement le prix moyen sur 7 jours, qui est contenu dans l'avant dernier élément du tableau
    card_average_price = frame_infos[-2].text.strip()
    return card_average_price.replace(',', '.').replace(' €', '').replace('£', '');

def getBuyableInfo(card):
    cards_sales = []

    # Récupération du contenu de la page
    url = deck[card][1]
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Recherche de la liste des cartes
    table = soup.find('div', {'class': 'table-body'})

    #dans table, trouver chaque ligne représentant une carte à vendre avec son prix, sa qualité et sa langue

    for row in table:
        cards_sales.append(row)

    # Stocker les informations de chaque carte à vendre en prenant son prix, sa qualité et sa langue
    for card in cards_sales:
        card_price = card.find('span', {"class" : "font-weight-bold color-primary small text-right text-nowrap"}).text.strip()[:-2]
        card_quality = card.find('span', {"class" : "badge"}).text.strip()
        card_language = card.find_all('span', {'class' : 'icon mr-2'})[-1].span['data-original-title']
        if card_quality in qualities:
            return [card_price.replace(',', '.'), card_quality, card_language]
    return ['0', '0', '0']

In [56]:
cards_sales = []

# Récupération du contenu de la page
url = deck["Jodah, the Unifier"][1]
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Recherche de la liste des cartes
table = soup.find('div', {'class': 'table-body'})

#dans table, trouver chaque ligne représentant une carte à vendre avec son prix, sa qualité et sa langue

for row in table:
    cards_sales.append(row)

# Stocker les informations de chaque carte à vendre en prenant son prix, sa qualité et sa langue
for card in cards_sales:
    #print((card.find_all('span', {'class' : 'icon mr-2'})[-1]).span['data-original-title'])
    card_price = card.find('span', {"class" : "font-weight-bold color-primary small text-right text-nowrap"}).text.strip()[:-2]
    card_quality = card.find('span', {"class" : "badge"}).text.strip()
    span_lg = card.find_all('span', {'class' : 'icon mr-2'})[-1]
    if span_lg.span is not None:
        card_language = span_lg.span['data-original-title']
    else:
        print("pas trouvé dans ")
        print(span_lg)
    
# prendre le dernier élément de card.find_all('span', {'class' : 'icon mr-2'})
# prendre le span['data-original-title'] de ce dernier élément


TypeError: 'NoneType' object is not subscriptable

In [46]:
print(getBuyableInfo('Jodah, the Unifier'))

TypeError: 'NoneType' object is not subscriptable

A présent que nous avons pu créer notre nouveau dictionnaire avec les cartes sélectionnées, nous allons pouvoir créer un nouveau fichier csv qui contiendra uniquement les infos des cartes sélectionnées et avoir plus rapidement les informations afin de pouvoir réagir presque en temps réel. Les cartes que j'ai sélectionnés sont purement arbitraire, je prévois de les inclure dans l'un de mes decks.

In [16]:
def getCardInfoWL(card):
    url = card_url[card]
    if(get_status_code(url) == 200):
        card_intel = []
        card_intel.append(getCardName(card))
        card_intel.append(getCardPrices(card))
        card_intel.append(getCardQuantity(card))
        card_intel.append(getCardAveragePrice(card))

    else:
        print("Le code d'état de la page de ", card, " est ", get_status_code(url), ", la page n'a pas pu être chargée.")
    return card_intel

#for card in card_url:
#    print(getCardInfoWL(card))

In [17]:
# à décommenter pour créer le fichier .csv (prends environ 1m20 max)
# Création du dossier s'il n'existe pas déjà
if not os.path.exists('data'):
    os.makedirs('data')

# Création du fichier .csv
with open('data/wants.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile, delimiter=';')
    # nommer les collonnes du tableau
    writer.writerow(['Card Name', 'Prices', 'Quantity', 'Average Price (7 days)'])
    for card in card_url:
        writer.writerow(getCardInfoWL(card))